This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.

In [182]:
import platform
import os
import pandas
import glob
import numpy

In [183]:
from parsing_functions import *

In [184]:
# Different locations if running either on Linux or Windows
if 'Linux' in platform.system():
    BasePath = os.path.join(os.path.sep, 'home', 'habi', 'P')
elif 'Windows' in platform.system():
    BasePath = os.path.join('P:', os.sep)
# Use *this* folder for the bone microvasculature manuscript
Root = os.path.join(BasePath, 'Documents', 'Publications', 'Ruslan Bone', 'manubot', 'content', 'data')
print('We are loading all the data from the folder %s' % Root)

We are loading all the data from the folder /home/habi/P/Documents/Publications/Ruslan Bone/manubot/content/data


In [185]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [186]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [187]:
print('We found %s log files in %s' % (len(Data), Root))

We found 54 log files in /home/habi/P/Documents/Publications/Ruslan Bone/manubot/content/data


Parse the data from all the log files

In [188]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['ControlSoftware'] = [controlsoftware(log) for log in Data['LogFile']]

In [189]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [pixelsize(log,rounded=True) for log in Data['LogFile']]

In [190]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]
Data['Exposure'] = [exposuretime(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [191]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [192]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumProj'] = [numproj(log) for log in Data['LogFile']]
Data['ProjSize'] = [projection_size(log) for log in Data['LogFile']]
Data['ThreeSixty'] = [threesixtyscan(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]
Data['Duration'] = [duration(log) for log in Data['LogFile']]
Data['Date'] = [scandate(log) for log in Data['LogFile']]

In [193]:
Data['NRecon'] = [nreconversion(log)[1] for log in Data['LogFile']]
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['GrayValue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]

In [194]:
Data['RecSize'] = [reconstruction_size(log) for log in Data['LogFile']]
Data['ROI'] = [region_of_interest(log, verbose=False) for log in Data['LogFile']]

In [195]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]
Data['Date'] = [scandate(log) for log in Data['LogFile']]

----

Now that we loaded the data, we customize the standard log file parser notebook for the microvasculature manuscript.

In [196]:
# Extract folder name
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [197]:
# Generate sample name to then match to figure number
# We bluntly split the path at the `os.path.sep` and user the first item of this separated list
Data['Sample'] = [(foldername[len(Root)+1:]).split(os.path.sep)[0] for foldername in Data['Folder']]

In [198]:
def match_to_figure(samplename):
    figurenumber = None
    if 'Israel' in samplename:
        figurenumber = 1
    elif '11O' in samplename:
        # Some are named 11OKT, some 11Okt
        figurenumber = 2
    elif 'c1m5' in samplename:
        figurenumber = 3
    elif '0_99' in samplename:
        figurenumber = 4    
    elif 'Vreni' in samplename:
        figurenumber = 5
    elif '_sample4' in samplename:
        figurenumber = 6
    elif 'Mouse_1EAR' in samplename:
        figurenumber = 7
    return(str(figurenumber))

In [199]:
# Map sample names to our figures 
Data['Figure'] = [match_to_figure(s) for s in Data['Sample']]

In [200]:
# Sort dataframe by figure number
Data.sort_values(by=['Figure', 'Sample'], inplace=True)
# Reset dataframe index
Data = Data.reset_index(drop=True)

In [201]:
# Cull irrelevant beginning of path from logfile
Data['LogFile'] = [lf[len(Root)+1:] for lf in Data['LogFile']]

In [202]:
# Cull irrelevant beginning of path from logfile
Data['Folder'] = [folder.split('data')[1] for folder in Data['Folder']]

In [203]:
Data.sample()

,LogFile,Scanner,ControlSoftware,Voxelsize,Voxelsize_rounded,Source,Camera,Exposure,Averaging,Voltage,...,NRecon,RingRemoval,Beamhardening,DefectPixelMasking,GrayValue,RecSize,ROI,Folder,Sample,Figure
37,Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET_...,SkyScan 1273,1.1,9.000382,9.0,Hamamatsu L9181-02,"DEXELA-2315[v1], S/N 32960",225,5,100.0,...,None,NaN,NaN,None,NaN,"(None, None)",False,/Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET...,Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET_,5


In [204]:
# Save out relevant columns of the dataframe as csv for adding to the supplementary materials
# With renamed column names
Data[['Figure', 'Sample', 'Scanner', 'ControlSoftware', 'Date',
      'Voxelsize_rounded', 'Source', 'Voltage', 'Current', 'Filter', 'ProjSize', 'ThreeSixty', 'RotationStep',
      'Averaging', 'Exposure', 'Stacks', 'Wide', 'Duration',
      'NRecon', 'RingRemoval', 'Beamhardening', 'LogFile'
     ]].to_csv(os.path.join(Root, 'ScanningDetails.csv'),
               index=False,
               header=['Figure', 'Sample name', 'Scanner', 'Control software version', 'Scan date',
                       'Voxelsize [μm]', 'X-ray source', 'Source voltage [kV]', 'Source current [μA]',
                       'Filter', 'Projection size', '360°-scan', 'Rotation step [°]', 'Frame averaging',
                       'Exposure time [ms]', 'Stacked scans', 'Overlap scans',
                       'Scan duration [s]',
                       'NRecon version', 'Ring removal correction', 'Beam hardening correction',
                       'Log file'
                     ])
# This csv file is nicely shown online on GitHub

In [205]:
# Save out as .xlsx sheet, too
# This Excel sheet is not uploaded to Github, but makes it easy to quickly look at it
Data[['Figure', 'Sample', 'Scanner', 'ControlSoftware', 'Date',
      'Voxelsize_rounded', 'Source', 'Voltage', 'Current', 'Filter', 'ProjSize', 'ThreeSixty', 'RotationStep',
      'Averaging', 'Exposure', 'Stacks', 'Wide', 'Duration',
      'NRecon', 'RingRemoval', 'Beamhardening', 'LogFile'
     ]].to_excel(os.path.join(Root, 'ScanningDetails.xlsx'),
               index=False,
               header=['Figure', 'Sample name', 'Scanner', 'Control software version', 'Scan date',
                       'Voxelsize [μm]', 'X-ray source', 'Source voltage [kV]', 'Source current [μA]',
                       'Filter', 'Projection size', '360°-scan', 'Rotation step [°]', 'Frame averaging',
                       'Exposure time [ms]', 'Stacked scans', 'Overlap scans',
                       'Scan duration [s]',
                       'NRecon version', 'Ring removal correction', 'Beam hardening correction',
                       'Log file'
                     ])

----
Now that we loaded all the relevant data from the log files, we can produce some text.
Copy-paste this text into the manuscript and edit accordingly.

In [210]:
Data.Folder.unique()

array(['/Mouse_Israel_C2_2.98um_LeftLeg/proj',
       '/EDTA3_Tage_11OKT17_Al025um_3_18um/proj',
       '/Maus_Hinterbein1_thirdScan_EDTA3plus4Tage_11Okt2017_Al025um_3_18um/proj',
       '/Maus_Hinterbein_11OKT17_Al025um_3_18um/proj',
       '/c1m5_1_65um_al025/proj', '/c1m5_1_65um_al025/rec',
       '/Mouse_kiefer_0_99um_VOI/VOI_01',
       '/Mouse_kiefer_0_99um_VOI/VOI_02',
       '/Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET_/proj',
       '/Vreni_Jaw_Jul19_AL1mm-Cu02mm_100kV_9um_OFFSET_/proj/Rec 03NOV2020',
       '/Vreni_Jaw_Jul20_AL1mm-Cu02mm_100kV_21um/proj',
       '/Vreni_Jaw_Jul20_AL1mm-Cu02mm_100kV_21um/rec',
       '/Ruslan_sample4_2214/proj', '/Ruslan_sample4_2214/rec',
       '/Mouse_1EAR_05SEP2023_molars_1_4um_50kV_CCD_nof/proj',
       '/Mouse_1EAR_05SEP2023_molars_1_4um_90kV_CCD_Al05/proj'],
      dtype=object)

In [217]:
# Print relevant data for each figure
for figure in Data.Figure.unique():
    print(40*'v', figure, 40*'v')
    print('The %s scans for Figure %s' % (len(Data[Data.Figure == figure]), figure), end=' ')
    print('were performed on a %s:' % Data[Data.Figure == figure].Scanner.unique(), end=' ')
    print('with control software version %s.' % Data[Data.Figure == figure].ControlSoftware.unique())
    print('The scans are:')
    for folder in Data[Data.Figure == figure].Folder:
        print('- %s' % folder)
    print('The X-ray source was set to a tube voltage of',
          " OR ".join(str(value) for value in Data[Data.Figure == figure].Voltage.unique()),
          'kV and a tube current of',
          " OR ".join(str(value) for value in Data[Data.Figure == figure].Current.unique()),
          'µA', end='')
    if Data[Data.Figure == figure].Filter.unique()[0]:  
        print(', the x-ray spectrum was filtered by',
              " OR ".join(str(value) for value in Data[Data.Figure == figure].Filter.unique()),
              'prior to incidence onto the sample.')
    else:
        print('.')
    print('For each scan, we acquired %s projections.' % Data[Data.Figure == figure].NumProj.unique(), end=' ')
    print('Projection images were recorded over a sample rotation of', end=' ')
    if Data[Data.Figure == figure].ThreeSixty.unique():
        print('360°', end=', ')
    else:
        print('180°', end=', ')
    print('with one projection acquired at every %s° and' % Data[Data.Figure == figure].RotationStep.unique(), end=' ')
    print('%s projections being averaged for noise reduction.' % Data[Data.Figure == figure].Averaging.unique())
    if len(Data[Data.Figure == figure].Wide.unique()) > 1:
        print('%s projections were stitched to cover the full extent of the sample' % Data[Data.Figure == figure].Wide.unique())
    else:
        if Data[Data.Figure == figure].Wide.unique():
            print(Data[Data.Figure == figure].Wide.unique())
    print('Each projection image with a size of %s pixels' % Data[Data.Figure == figure].ProjSize.unique(), end= ' ')
    if len(Data[Data.Figure == figure].Exposure.unique()) > 1:
        print('was exposed for %s ms (on average).' % (round(numpy.mean(Data[Data.Figure == figure].Exposure.unique()))))
    else:
        print('was exposed for %s ms.' % Data[Data.Figure == figure].Exposure.unique())
    print('This resulted in datasets with an isometric voxel size of %s μm.' % Data[Data.Figure == figure].Voxelsize.unique())
    print(40*'^', figure, 40*'^')

vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv 1 vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
The 3 scans for Figure 1 were performed on a ['SkyScan 1172 (Version F)']: with control software version ['1.5 (build 26)'].
The scans are:
- /Mouse_Israel_C2_2.98um_LeftLeg/proj
- /Mouse_Israel_C2_2.98um_LeftLeg/proj
- /Mouse_Israel_C2_2.98um_LeftLeg/proj
The X-ray source was set to a tube voltage of 49.0 kV and a tube current of 200.0 µA.
For each scan, we acquired [7202] projections. Projection images were recorded over a sample rotation of 360°, with one projection acquired at every [0.05]° and [3] projections being averaged for noise reduction.
Each projection image with a size of [(4000, 2672)] pixels was exposed for [985] ms.
This resulted in datasets with an isometric voxel size of [2.99] μm.
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 1 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv 2 vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
The 17 scans for Figure 2 were perform